In [1]:
import stardog
from SPARQLWrapper import SPARQLWrapper, JSON, XML
from pymantic import sparql
from datetime import  datetime
import numpy as np
import pandas as pd

In [2]:
connection_details = {
  'endpoint': 'http://localhost:5820',
  'username': 'admin',
  'password': 'admin'
}

In [8]:
with stardog.Admin(**connection_details) as admin:
    for db in admin.databases():
        print(db.name)

In [8]:
database_name='RDF_star_based_approach'
Time_Dictionary=dict()
average_dict=dict()
stdv_dic=dict()

# Defining Functions

In [1]:
def calculate_time(query,variable_name):
    start_time = datetime.now()
    conn.select(query, timeout=0)
    end_time = datetime.now()
    time=(round((end_time- start_time ).total_seconds()*1000))
    add_to_list(variable_name,time)   
    
def add_to_list(variable_name,time):
    if variable_name in Time_Dictionary:
        Time_Dictionary[variable_name].append(time)
    else:
        Time_Dictionary.update({variable_name:[]})
        Time_Dictionary[variable_name].append(time)

def change_name_index(value):
    value=value+1
    result= 'round '+ str(value)
    return result

def create_dataframe(dictionary):
    for key in dictionary:
        average_dict.update({key:[]})
        average_dict[key].append(sum(dictionary[key])/len(dictionary[key]))
    for key in dictionary:
        stdv_dic.update({key:[]})
        stdv_dic[key].append(np.std(dictionary[key]))
    result = pd.DataFrame.from_dict(dictionary)
    for index, row in result.iterrows():
        result.rename(index={index:change_name_index(index)},inplace=True)
    average=pd.DataFrame.from_dict(average_dict)
    average.index=['Average']
    stdv=pd.DataFrame.from_dict(stdv_dic)
    stdv.index=['stdv']
    frames=[result, average, stdv]
    final_result=pd.concat(frames)
    write_csv(final_result)

def write_csv(result):
    result.to_csv('RDF-star-based-approach_Stardog.csv')

In [10]:
conn=stardog.Connection(database_name,**connection_details)

In [3]:
conn.begin()

In [2]:
print(conn.select('select (count(*) as ?c) where{?n ?m ?l.}'))

In [13]:
Start_Proces=datetime.now()

In [ ]:
for i in range(10):
    print('round '+ str(i+1) +' started')
    query1="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?id ?lbl  
        WHERE{  
            ?n1 rdfs:label ?lbl.  
            ?n1 ns1:name 'XU JIE'.  
            OPTIONAL{?n1 ns1:node_id ?id.} 
        }       
   """
    calculate_time(query1,"query1")

    query2="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?lbl ?country 
        WHERE{ 
            ?n rdfs:label ?lbl. 
            ?n ns1:name 'EL PORTADOR'. 
            ?n ns1:countries ?country. 
            FILTER(?country!='') 
        } 
    """ 
    calculate_time(query2,"query2")

    query3="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?lbl  
        WHERE{  
            ?n rdfs:label ?lbl.  
        }        
    """
    calculate_time(query3,"query3")

    query4="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?lbl   
        WHERE{ 
            ?n ?lbl ?n1.   
            FILTER(! IsLiteral(?n1)) 
        }       
    """
    calculate_time(query4,"query4")
 
    query5="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT  ?name ?lbl  
        WHERE{     
            ?n1 ns1:name 'ORION HOUSE SERVICES (HK) LIMITED'. 
            ?n1 ns1:intermediary_of ?n2. 
            ?n2 rdfs:label ?lbl. 
            OPTIONAL{?n2 ns1:name ?name.} 
        }         
    """
    calculate_time(query5,"query5")
  
    query6="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?lbl1 ?name ?link ?lbl2 
        WHERE{   
              {   
                ?n1 rdfs:label ?lbl1.  
                ?n1 ns1:officer_of ?n2.  
                ?n2 rdfs:label ?lbl2. 
                ?n1 ns1:name 'BEARER'. 
                OPTIONAL{         
                ?n2 ns1:name ?name.} 
                OPTIONAL { <<?n1 ns1:officer_of ?n2>> ns1:link ?link. }       
            }  
            UNION{  
                ?n1 rdfs:label ?lbl1.  
                ?n1 ns1:officer_of ?n2.  
                ?n2 rdfs:label ?lbl2.  
                ?n1 ns1:name 'The Bearer'. 
                OPTIONAL { ?n2 ns1:name ?name. } 
                OPTIONAL { <<?n1 ns1:officer_of ?n2>> ns1:link ?link. }      
            }  
        }          
    """
    calculate_time(query6,"query6")

    query7="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?lbl1 ?link ?lbl2  ?entity_name 
        WHERE{  
            {<<?n1 ?r1 ?n2>> ns1:end_date '11-APR-2014'.  
            ?n1 rdfs:label ?lbl1.  
            ?n2 rdfs:label ?lbl2.  
            ?n2 ns1:status 'Active'.  
            ?n1 ns1:name 'THE BEARER'. 
            OPTIONAL { <<?n1 ?r1 ?n2>> ns1:link ?link. }              
            OPTIONAL { ?n2 ns1:name ?entity_name. } 
            }  
            UNION  
            {<<?n1 ?r1 ?n2>> ns1:end_date '29-DEC-2009'.  
            ?n1 rdfs:label ?lbl1.  
            ?n2 rdfs:label ?lbl2.  
            ?n2 ns1:status 'Defaulted'.  
            ?n1 ns1:name 'THE BEARER'. 
            OPTIONAL { <<?n1 ?r1 ?n2>> ns1:link ?link. }  
            OPTIONAL { ?n2 ns1:name ?entity_name. } 
            }  
        }    
    """
    calculate_time(query7,"query7")
    
    query8="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?lbl1 ?name ?link ?address  
        WHERE{  
              ?n1 rdfs:label ?lbl1.  
              ?n1 ns1:registered_address ?n3.  
              OPTIONAL { ?n1 ns1:name ?name.} 
              OPTIONAL { <<?n1 ns1:registered_address ?n3>> ns1:link ?link. }  
              OPTIONAL { ?n3 ns1:address ?address. }     
        }      
    """
    calculate_time(query8,"query8")

    query9="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?lbl ?name ?link1 ?first_address ?link2 ?second_address  
        WHERE { 
            ?n1 ns1:registered_address ?n2. 
            ?n1 ns1:registered_address ?n3. 
            ?n1 rdfs:label ?lbl. 
            OPTIONAL { ?n1 ns1:name ?name. } 
            OPTIONAL { <<?n1 ns1:registered_address ?n2>> ns1:link ?link1. } 
            OPTIONAL { ?n2 ns1:address ?first_address. } 
            OPTIONAL { <<?n1 ns1:registered_address ?n3>> ns1:link ?link2. } 
            OPTIONAL { ?n3 ns1:address ?second_address. } 
            FILTER(?n2 != ?n3) 
        }        
    """
    calculate_time(query9,"query9")
    
    query10="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?link1 ?link2 ?address 
        WHERE{ 
                ?n1 rdfs:label 'officer'. 
                ?n1 ns1:registered_address ?n3. 
                ?n2 rdfs:label 'entity'. 
                ?n2 ns1:registered_address ?n3. 
               OPTIONAL { <<?n1 ns1:registered_address ?n3>> ns1:link ?link1.} 
               OPTIONAL { <<?n2 ns1:registered_address ?n3>> ns1:link ?link2.} 
               OPTIONAL { ?n3 ns1:address ?address. } 
        }        
    """
    calculate_time(query10,"query10")

    query11="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?name ?valid ?sourceID 
        WHERE{ 
             ?n1 rdfs:label 'intermediary'. 
             ?n4 rdfs:label 'officer'.  
             ?n2 rdfs:label 'entity'. 
             ?n3 rdfs:label 'address'. 
             ?n1 ?r1 ?n2. 
             ?n2 ?r2 ?n3.    
             ?n4 ?r3 ?n3.  
             OPTIONAL { <<?n1 ?r1 ?n2>> ns1:valid_until ?valid. } 
             OPTIONAL { <<?n1 ?r1 ?n2>> ns1:sourceID ?sourceID. } 
             OPTIONAL {?n1 ns1:name ?name.} 
        }       
    """
    calculate_time(query11,"query11")
    
    query12="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?officer_name ?link1 ?officer2_name ?link2 ?entity_name ?intermediary_name ?valid_until  ?e_name  
        WHERE{ 
            ?n1 rdfs:label 'officer'. 
            ?n2 rdfs:label 'officer'. 
            ?n3 rdfs:label 'address'. 
            ?n4 rdfs:label 'entity'. 
            ?n5 rdfs:label 'intermediary'. 
            ?n6 rdfs:label 'entity'. 
            ?n1 ns1:officer_of ?n2. 
            ?n2 ns1:registered_address ?n3. 
            ?n2 ns1:officer_of ?n4. 
            ?n5 ns1:intermediary_of ?n4. 
            ?n5 ns1:intermediary_of ?n6. 
            OPTIONAL{?n1 ns1:name ?officer_name.} 
            OPTIONAL{<<?n1 ns1:officer_of ?n2>> ns1:link ?link1.} 
            OPTIONAL{?n2 ns1:name ?officer2_name.}  
           OPTIONAL{<<?n2 ns1:officer_of ?n4>> ns1:link ?link2.}  
           OPTIONAL{?n4 ns1:name ?entity_name.}  
           OPTIONAL{?n5 ns1:name ?intermediary_name.} 
           OPTIONAL{<<?n5 ns1:intermediary_of ?n4>> ns1:valid_until ?valid_until.} 
           OPTIONAL{?n6 ns1:name ?e_name.}  
            FILTER(?n4!=?n6) 
        }       
    """
    calculate_time(query12,"query12")
    print('round '+ str(i+1) +' finished')

In [9]:
End_Process=datetime.now()

In [ ]:
Total_Time_Process=End_Process-Start_Proces

In [ ]:
create_dataframe(Time_Dictionary)

In [ ]:
print(round((Total_Time_Process ).total_seconds()*1000))